# 1 -  Planejamento da Solução

## Entendimento de Negócio

**Qual é o problema de negócio?**

- Empresa alimentícia situada no RJ, deseja abrir filiais na cidade de São Paulo. Para isso, precisa de 3 análises:
    - 1 - Estimar faturamento que uma loja teria em cada um dos bairros de São Paulo (regressão, neste caso sem séries temporais).
    - 2 - Classificar o potencial dos bairros de São Paulo em alto, médio ou baixo (classificação multiclasse).
    - 3 - Segmentar os bairros de São Paulo de acordo com o perfil de renda e idade, identificando os com maior aderência ao público alvo.
    
- Público alvo: adultos de 25 a 50 anos, das classes A (rendas A1 e A2) e B (rendas B1 e B2). 

**Saída**

- Prototipagem técnica da solução: https://docs.google.com/spreadsheets/d/17lxCBRLPEuNCO25BimVFRE3Tms-314WSTpJ1xgEIf38/edit#gid=0

- O que será entregue, efetivamente? / Onde o time de negócio quer ver?
    - Documento no formato doc, pdf ou ppt, voltado ao negócio, apresentando um racional de como os dados foram analisados. Detalhar com gráficos, tabelas, e descrever conclusões.
    - Responder: Dada a natureza do problema apresentado, que outro dado externo (fontes públicas ou privadas) poderia ser utilizado para agregar mais valor ao resultado? Por que?
    
**Entrada**
- Fontes de dados:
    - Dataset contendo faturamento e potencial dos bairros do Rio de Janeiro do cliente, bem como dados sociodemográficos do bairros do Rio de Janeiro e São Paulo.

- Ferramentas:
    - Python 3.8.12, Jupyter Notebook, Git, Github.

**Processamento**
- Tipo de problema: análise exploratória, regressão, classificação e clusterização.
- Metodologia: CRISP-DM, metodologia ágil (iterativa e incremental) para desenvolvimento de projetos de ciência de dados.


## Implementado nesta Sprint

**Ciclo 6**

Desenvolvimento da Análise 1 - versão 2 (regressão):
- Correção de split nos dados.


---------------------------------
- Split dos dados.
- Modelagem de dados (data preparation).
- Seleção de Features.
- Model Based feature selection.
- Novo KNN Regressor + comparação com o baseline do ciclo 1.
- KNN Regressor com Random Search + Cross validation.
- Novos algoritmos de ML: Lasso (linear), XGBoost Reegressor (embedding de trees).
- Comparação da performance dos modelos.
- Performance de generalização do melhor modelo com dados de teste.
- Modelos final, treinado com dataset traino + validação, prevendo faturamento por bairro em SP (produção).

# 2 - Importações

## Bibliotecas

In [3]:
import pandas                      as pd 
import seaborn                     as sns
import numpy                       as np
import sweetviz                    as sv
import xgboost                     as xgb
import inflection
import warnings

from IPython.core.display          import HTML
from matplotlib                    import pyplot as plt
from tabulate                      import tabulate
from matplotlib.ticker             import FuncFormatter
from IPython.display               import Image

from sklearn.model_selection       import train_test_split, GridSearchCV
from sklearn.neighbors             import KNeighborsRegressor
from sklearn.metrics               import r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing         import MinMaxScaler, OrdinalEncoder
from sklearn.feature_selection     import SelectFromModel, SelectKBest, f_regression
from sklearn.ensemble              import RandomForestRegressor
from sklearn.linear_model          import Lasso

## Funções Auxiliares

In [2]:
def jupyter_settings():
    """ Otimiza configurações gerais, padronizando tamanhos de plots, etc """
    %matplotlib inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [20, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.set_option('display.max_columns', 30)
    pd.set_option('display.max_rows', 30)
    pd.set_option('display.expand_frame_repr', False )
    pd.set_option('display.float_format', lambda x: '%.4f' % x)
    pd.set_option('max_colwidth', None)
    sns.set()
jupyter_settings()

def ml_error( model_name, y, yhat ):
    """ Calcula os erros do modelo de regressão recebido 
    model_name: nome do modelo
             y: valores de reais
          yhat: valores de estimados pelo modelo """
    mae = mean_absolute_error (y, yhat)
    mape = mean_absolute_percentage_error( y, yhat )
    rmse = np.sqrt( mean_squared_error( y, yhat ) )
    
    return pd.DataFrame( {'Model Name': model_name,
                          'MAE': mae,
                          'MAPE': mape,
                          'RMSE': rmse  }, index=[0] )

# 3 - Análise 1: Estimativa de faturamento em SP

Análise 1 - Estimar o faturamento que uma loja teria em cada um dos bairros de São Paulo (regressão).

## Split dos Dados

In [4]:
df_rj = pd.read_csv('../data/interim/df_rj_feat_eng_done.csv', index_col=0)
print(df_rj.shape)
df_rj.head(1)

(160, 22)


,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,faturamento,potencial
0,11676,1027,483,688,800,1675,2300,1784,2919,3975,0,145,715,1242,1093,758,92,304,2102,2501.0000,932515,Médio


A análise 1 é um problema de regressão (supervisionado), sendo a variável alvo "faturamento" do tipo float (contínua).

Neste ciclo 6, df_rj será dividida em duas partes, visando o GridSearchCV para cross validation e hiperparâmetros:
- Dados de treinamento + validação (trainval): utilizados para treinar os modelos de machine learning, para validar a performance dos modelos, e para tunagem de hiperparâmetros.
- Dados de teste (test): utilizados para avaliação final da capacidade de generalização dos modelos, simulando dados de produção (registros de SP).

Divisão entre features e variável alvo:

In [5]:
y1 = df_rj.faturamento
y1.head(2)

0    932515
1    588833
Name: faturamento, dtype: int64

In [6]:
X1 = df_rj.drop(["faturamento"], axis=1)
X1.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
0,11676,1027,483,688,800,1675,2300,1784,2919,3975,0,145,715,1242,1093,758,92,304,2102,2501.0000,Médio


Divisão entre datasets de treinval e teste, mantendo 20% dos registros para teste.
O parâmetro "random_state" é utilizado para manter a reprodutibilidade da divisão.

In [7]:
X1_trainval, X1_test, y1_trainval, y1_test = train_test_split(X1, y1, random_state=0, test_size=0.20)

Conferência das divisões:

In [8]:
print(X1_trainval.shape)
print(X1_test.shape)
X1_trainval.head(1)

(128, 21)
(32, 21)


,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
54,13480,1905,933,1140,1233,2195,2879,1491,1704,5074,0,0,91,480,1334,1253,240,1025,571,1133.0000,Baixo


## Preparação dos Dados

Os algoritmos de machine learning possuem premissas com relação ao formato dos dados que recebem. Elas devem ser respeitadas sempre que possível, para que o algoritmo atinja a maior performance possível.

Nesta seção, as features serão preparadas para servir de insumo aos algoritmos.

In [9]:
X1_trainval.columns

Index(['populacao', 'pop_ate9', 'pop_de10a14', 'pop_de15a19', 'pop_de20a24',
       'pop_de25a34', 'pop_de35a49', 'pop_de50a59', 'pop_mais_de60',
       'pop_alvo', 'domicilios_a1', 'domicilios_a2', 'domicilios_b1',
       'domicilios_b2', 'domicilios_c1', 'domicilios_c2', 'domicilios_d',
       'domicilios_e', 'domicilios_alvo', 'renda_media', 'potencial'],
      dtype='object')

In [10]:
X1_trainval.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
54,13480,1905,933,1140,1233,2195,2879,1491,1704,5074,0,0,91,480,1334,1253,240,1025,571,1133.0000,Baixo


Tratativas necessárias para as features da análise 1 (regressão).

- Numéricas (tratadas com scaling): 'populacao', 'pop_ate9', 'pop_de10a14', 'pop_de15a19', 'pop_de20a24', 'pop_de25a34', 'pop_de35a49', 'pop_de50a59', 'pop_mais_de60', 'pop_alvo', 'domicilios_a1', 'domicilios_a2', 'domicilios_b1', 'domicilios_b2', 'domicilios_c1', 'domicilios_c2', 'domicilios_d', 'domicilios_e', 'domicilios_alvo', e 'renda_media'. 
- Categóricas (tratadas com encoding): 'potencial'.
- Variável resposta (tratada com transformação logaritmica): 'faturamento'.

### Scaling

1 - Feature Scaling: Trazer todas variáveis para mesmo range (escala), para não enviesar os modelos.
- 1.1 Standardization se distribuição Normal:
    - Sem outliers: usa StandardScaler
    - Com outliers: usa RobustScaler
- 1.2 Normalization se não normal:
    - Com ou sem outliers: Usar MinMaxScaler

Avaliando as distribuições das features na seção "Estatística Descritiva", nenhuma das que serão tratatas aqui possui distribuição normal. Logo, todas serão reescaladas usando a classe "MinMaxScaler" do scikit-learn.
- MinMaxScaler: Rescala para o intervalo entre 0 e 1.

In [11]:
features_scaling = ['populacao', 'pop_ate9', 'pop_de10a14', 'pop_de15a19', 'pop_de20a24', 'pop_de25a34',
'pop_de35a49', 'pop_de50a59', 'pop_mais_de60', 'pop_alvo', 'domicilios_a1', 'domicilios_a2', 'domicilios_b1',
'domicilios_b2', 'domicilios_c1', 'domicilios_c2', 'domicilios_d', 'domicilios_e', 'domicilios_alvo', 'renda_media']

Reescala apenas as features selecionadas do trainval com MinMaxScaler.

In [12]:
scaler1 = MinMaxScaler()
X1_trainval[features_scaling] = scaler1.fit_transform(X1_trainval[features_scaling])
X1_trainval.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
54,0.0199,0.0153,0.0165,0.0179,0.0201,0.0252,0.0270,0.0178,0.0185,0.0262,0.0000,0.0000,0.0047,0.0226,0.0401,0.0485,0.0484,0.0690,0.0120,0.0076,Baixo


Aplicar desde já a reescala também no test test.

In [13]:
X1_test[features_scaling] = scaler1.transform(X1_test[features_scaling])
X1_test.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
110,0.0179,0.0121,0.0122,0.0148,0.0146,0.0210,0.0249,0.0181,0.0219,0.0232,0.0000,0.0080,0.0171,0.0266,0.0324,0.0406,0.0373,0.0577,0.0208,0.0164,Baixo


Features Reescaladas.

### Transformations

2 - Transformação:
- 2.1 - Converter variáveis categóricas em numéricas (Encoding).
- 2.2 - Converter a variável resposta em distribuição normal.(Response Variable Transf.)

#### Encoding

2.1 - Encoding:
- É a Conversão de Features Categóricas para Numéricas, mantendo o conteúdo.
- Avaliando a feature "potencial", há uma relação de ordem na variável, que contém valores "Baixo", "Médio" e "Alto". Em função disso, será aplicando um encoding ordinal através da classe "OrdinalEncoder" do scikit-learn.

Encodar apenas as features selecionadas do treino com OrdinalEncoder.

In [14]:
encoder1 = OrdinalEncoder(categories=[['Baixo', 'Médio', 'Alto']], dtype=int)
X1_trainval.potencial = encoder1.fit_transform(X1_trainval.potencial.to_numpy().reshape(-1,1))
encoder1.categories_
X1_trainval.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
54,0.0199,0.0153,0.0165,0.0179,0.0201,0.0252,0.0270,0.0178,0.0185,0.0262,0.0000,0.0000,0.0047,0.0226,0.0401,0.0485,0.0484,0.0690,0.0120,0.0076,0


Aprontar também o test, já aplicando as transformações.

In [15]:
X1_test.potencial = encoder1.transform(X1_test.potencial.to_numpy().reshape(-1,1))
X1_test.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
110,0.0179,0.0121,0.0122,0.0148,0.0146,0.0210,0.0249,0.0181,0.0219,0.0232,0.0000,0.0080,0.0171,0.0266,0.0324,0.0406,0.0373,0.0577,0.0208,0.0164,0


#### Response Variable Transformation

2.2 - Response Variable Transformation:
- Aplica um log na variável resposta, transformando ela em algo próximo a uma distribuição normal.

Será tratado em ciclos futuros. 

## Seleção de Features

Neste ciclo 6, apenas a seleção de features baseada em algoritmos será utilizada.

### Baseada em Algoritmos

Forma mais sofisticada, utilizar machine learning para julgar a importância das features, mantendo as mais importantes (model-based feature selection). 

Será utilizada a classe "SelectFromModel" do scikit-learn para isto.
- Neste método, um algoritmo precisa prover alguma medida de importância, para o rankeamento por esta medida.
    - Modelos baseados em árvores proveem "feature_importances_", que encoda a importância de cada feature.
    - Modelos lineares proveem "coefficients", que também podem ser utilizados para isto.
- Será utilizada uma RandomForestRegressor com 100 árvores, e threshhold com um fator de escala multiplicado pela mediana.
- Diferente da estatística univariada, com esta técnica é possível capturar interactions (features derivadas de outras), caso o modelo utilizado seja capáz de capturá-las.

In [16]:
sel_mbfs = SelectFromModel( RandomForestRegressor(n_estimators=10, random_state=0), threshold="0.6*median") 
sel_mbfs.fit(X1_trainval, y1_trainval).transform(X1_trainval)
features_mbfs = sel_mbfs.get_feature_names_out()
features_mbfs

array(['pop_ate9', 'pop_de10a14', 'pop_de15a19', 'pop_de20a24',
       'domicilios_a1', 'domicilios_a2', 'domicilios_b1', 'domicilios_b2',
       'domicilios_c2', 'domicilios_d', 'domicilios_e', 'renda_media'],
      dtype=object)

Mantendo apenas as features selecionadas nos demais datasets.

In [17]:
X1_trainval = X1_trainval[features_mbfs].copy() 
X1_test = X1_test[features_mbfs].copy() 
X1_trainval.head(1)

,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c2,domicilios_d,domicilios_e,renda_media
54,0.0153,0.0165,0.0179,0.0201,0.0000,0.0000,0.0047,0.0226,0.0485,0.0484,0.0690,0.0076


In [19]:
X1_test.head(1)

,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c2,domicilios_d,domicilios_e,renda_media
110,0.0121,0.0122,0.0148,0.0146,0.0000,0.0080,0.0171,0.0266,0.0406,0.0373,0.0577,0.0164


Na sequência, alguns modelos de machine learning serão experimentados.

As seções "Implementação de Baseline (Ciclo 1)" e "Comparação com o Baseline" do ciclo 3 foram removidas, visto que o split de dados foi realizado visando um GridSearchCV.

## KNN Regressor

A seguir, será implmentado um KNN Regressor com Grid Search + Cross Validation.

O objetivo do Grid Search (uma forma de fine tuning) é encontrar os melhores parâmetros para o modelo, visando uma maior perfomance. 
- Os possíveis parâmetros serão definidos abaixo, e um modelo será gerado para cada possibilidades de combinação pelo Grid Search.

O Cross Validation é um método estatístico que serve para avaliar a performance de modelos, considerando as diferentes possíveis divisões do dataset. 
- Abaixo, será realizada uma divisão de 5 partes no dataset, e obtido o cálculo do score médio de cada parte.

Dado o pequeno número de dados disponíveis, a métrica utilizada para avaliação do melhor modelo será o MAE:
- MAE (Erro absoluto médio): É a diferença entre os valores preditos e reais.
Atribui peso igual para todos os erros, logo "dilui o outlier", reduzindo seu peso. 
    - Significa: Na estimativa de faturamento dos bairros, o modelo erra em média R$ X,XX. 
- RMSE (Raiz quadrada do erro médio): Similar ao MAE, mas dá mais peso ao outliers.

In [181]:
knn_grid = {'n_neighbors':[2,3,4,5,6,7,8,9,10,11,12]}
#instanciar GridSearchCV com KNNRegressor e treinar o modelo
knn_reg = GridSearchCV(KNeighborsRegressor(), knn_grid, cv=5, scoring='neg_mean_absolute_error', #MAE
                            return_train_score=True).fit(X1_trainval, y1_trainval)

Analisar os melhores parâmetros do KNN. 

In [182]:
print(f"Melhores parâmetros: {knn_reg.best_params_}")

Melhores parâmetros: {'n_neighbors': 2}


Análise de performance sobre Treino e Validação:

In [183]:
df_knn_rank = pd.DataFrame(knn_reg.cv_results_)[['rank_test_score','params','mean_train_score','rank_test_score']].sort_values('rank_test_score')[:8]
df_knn_rank

,rank_test_score,params,mean_train_score,mean_test_score
0,1,{'n_neighbors': 2},-41579.9423,-73423.4488
1,2,{'n_neighbors': 3},-52534.1292,-74712.0003
2,3,{'n_neighbors': 4},-58027.6378,-81076.3062
3,4,{'n_neighbors': 5},-67246.6447,-84373.9878
4,5,{'n_neighbors': 6},-72474.9062,-92972.9324
5,6,{'n_neighbors': 7},-81603.3660,-99272.3818
6,7,{'n_neighbors': 8},-87486.9682,-107750.0150
7,8,{'n_neighbors': 9},-94885.1870,-112950.8689


Pela diferença entre train e val, percebe-se que houve overfitting.

Ao aumentar o número de K do KNN, o modelo fica mais simples, tendendo a overfitar menos.

Dado que isto não aconteceu, ao que tudo indica, a pouca quantidade de dados é o motivo do overfitting.

Cria dataframe de Scores e inclui o do KNN.

In [192]:
df_scores = pd.DataFrame(columns=['model_name','MAE'])
df_scores.loc[len(df_scores)] = ['knn_reg', df_knn_rank.loc[df_knn_rank.rank_test_score == 1,'mean_test_score'][0]]
df_scores

,model_name,MAE
0,knn_reg,-73423.4488


## Regressão Linear - Lasso

Será avaliado na sequência um modelo de regressão linear chamado Lasso.
- Como o dataset tem poucos registros, é imporante regularizar o modelo: reduzir sua complexidade, para evitar overfitting.
- Ele foi escolhido pois assume-se que só algumas features são importantes, logo cabe o uso da regularização L1 (implementada pelo Lasso). O Lasso ignora totalmente algumas features, revelando as mais importantes.
- Serão utilizados os parâmetros:
    - "alpha": controla a força com que os coeficientes são empurrados para zero. Aumentando, reduz-se o overfitting.
    - "max_iter": número máximo de iterações a serem executadas, precisa ser aumentado ao aumentar o alpha.

In [187]:
lasso_grid = {'alpha':[10, 100, 1000, 10000, 100000], 'max_iter':[100000]}
#instanciar GridSearchCV com Lasso e treinar o modelo
lasso_reg = GridSearchCV(Lasso(random_state=0), lasso_grid, cv=5, scoring='neg_mean_absolute_error', #MAE
                            return_train_score=True).fit(X1_trainval, y1_trainval)

Analisar os melhores parâmetros do Lasso. 

In [188]:
print(f"Melhores parâmetros: {lasso_reg.best_params_}")

Melhores parâmetros: {'alpha': 1000, 'max_iter': 100000}


Análise de performance sobre Treino e Validação:

In [197]:
df_lasso_rank = pd.DataFrame(lasso_reg.cv_results_)[['rank_test_score','params','mean_train_score','mean_test_score']].sort_values('rank_test_score')[:8]
df_lasso_rank

,rank_test_score,params,mean_train_score,mean_test_score
2,1,"{'alpha': 1000, 'max_iter': 100000}",-73641.2063,-93060.1781
1,2,"{'alpha': 100, 'max_iter': 100000}",-69039.3950,-113596.7019
0,3,"{'alpha': 10, 'max_iter': 100000}",-68733.9063,-121198.5077
3,4,"{'alpha': 10000, 'max_iter': 100000}",-120803.4771,-131227.5883
4,5,"{'alpha': 100000, 'max_iter': 100000}",-295615.0098,-296940.4055


Novamente o modelo overfitou, e mesmo tunando os hiperparâmetros, mais uma vez não foi possível resolver isto. 

Ao que tudo indica, novamente isto se deve pela pequena quantidade de registros. 

Salvar o score do Lasso.

In [199]:
df_scores.loc[len(df_scores)] = ['lasso_reg',df_lasso_rank.loc[df_lasso_rank.rank_test_score == 1,'mean_test_score'][2]]
df_scores

,model_name,MAE
0,knn_reg,-73423.4488
1,lasso_reg,-93060.1781


## XGBoost Regressor

O XGBoost (Extreme Gradient Boosting) é um ensembles de Decision Trees (árvores de decisão). Ensembles são métodos que combinam múltiplos modelos de machine learning para criar modelos ainda mais poderosos.
- Gradient Boostings estão entre os mais poderosos e amplamente utilizados modelos de ML de aprendizagem supervisionada.
- A principal vantágem do Gradient Boosting é combinar vários modelos simples (weak learners), como swallow trees (árvores rasas). Cada modelo só provê um bom resultado em parte dos dados, mas adicionando mais árvores, melhora-se a performance geral.
- São mais sensíveis a parâmetros que as Random Forests, mas podem prover maior acurácia, se parametrizados adequadamente.
- A escolha dos parâmetros e sua tunagem priorizou o controle de overfitting. 

In [289]:
#posso controlar com 'max_depth':[1], ou com 'min_child_weight':[15]

xgb_grid = {'max_depth':[2, 3],  'eta':[0.1, 0.2, 0.3] }
        #'n_estimators':[12, 25, 50], 'min_child_weight':[1, 3], 'colsample_bytree':[0.5, 1], 'eta':[0.5, 0.9]
#instanciar GridSearchCV com XGBRegressor e treinar o modelo
xgb_reg = GridSearchCV(xgb.XGBRegressor(), xgb_grid, cv=5, scoring='neg_mean_absolute_error', #MAE
                            return_train_score=True).fit(X1_trainval, y1_trainval)

Analisar os melhores parâmetros do XGBoost. 

In [292]:
print(f"Melhores parâmetros: {xgb_reg.best_params_}")

Melhores parâmetros: {'eta': 0.2, 'max_depth': 3}


Análise de performance sobre Treino e Validação:

In [293]:
df_xgb_rank = pd.DataFrame(xgb_reg.cv_results_)[['rank_test_score','params','mean_train_score','mean_test_score']].sort_values('rank_test_score')
df_xgb_rank

,rank_test_score,params,mean_train_score,mean_test_score
3,1,"{'eta': 0.2, 'max_depth': 3}",-3185.1910,-85509.3828
2,2,"{'eta': 0.2, 'max_depth': 2}",-10765.1343,-86081.4925
1,3,"{'eta': 0.1, 'max_depth': 3}",-12073.3094,-87755.0900
0,4,"{'eta': 0.1, 'max_depth': 2}",-22505.1664,-88335.8710
5,5,"{'eta': 0.3, 'max_depth': 3}",-977.0886,-90344.0457
4,6,"{'eta': 0.3, 'max_depth': 2}",-6149.4659,-92932.5767


Como no cenário anterior, o modelo overfitou mesmo tunando os hiperparâmetros.

Ao que tudo indica, novamente isto se deve pela pequena quantidade de registros.

Salvar o score do XGBoost.

In [300]:
df_scores.loc[len(df_scores)] = ['xgb_reg', df_xgb_rank.loc[df_xgb_rank.rank_test_score == 1,'mean_test_score'][3]]
df_scores.sort_values('MAE', ascending=False)

,model_name,MAE
0,knn_reg,-73423.4488
2,xgb_reg,-85509.3828
1,lasso_reg,-93060.1781


## Performance dos Modelos

A tabela abaixo detalha a melhor performance obtida com cada modelo de machine learning, já com fine tuning e cross validation.

In [301]:
df_scores.sort_values('MAE', ascending=False)

,model_name,MAE
0,knn_reg,-73423.4488
2,xgb_reg,-85509.3828
1,lasso_reg,-93060.1781


O menor erro MAE foi obitodo com o KNN, e ele será utilizado para a sequência da análise.

## Performance de Generalização

Agora, diferente do realizado até aqui, o modelo irá fazer a predição em cima dos dados de teste.

Desta forma, será possível avaliar a sua capacidade de generalização (prever dados inéditos), simulando dados de produção (SP). 

Os modelo utilizado será o de menor erro MAE nos dados de validação, e com os melhores parâmetros já identificados com cross validation.

Dado que o GridSearchCV já retreina um modelo usando os melhores parâmetros encontrados no dataset todo (treino + validação), ele será utilizado para avaliar os dados de teste.

In [305]:
knn_score_final = knn_reg.score(X1_test, y1_test)
knn_score_final

-44931.546875

O modelo teve um MAE consideravalmente menor no treino, com relação a validação.

Houve grande diferença entre os dados de treino e validação, bem como grande diferença entre os dados de validação do melhor modelo e de teste.

Tudo isso indica que a abordagem mais adequada para a solução deste problema é a bayesiana, ao invés da frequentista.

## Previsão Faturamento SP

### Carregar dados de produção

In [ ]:
Carregar dados de SP (produção).

In [339]:
df_sp_raw = pd.read_csv('../data/interim/df_sp_raw.csv', index_col=0)
X1_prod = df_sp_raw.copy()
print(X_prod.shape)
X1_prod.head(1)

(296, 12)


,codigo,bairro,cidade,estado,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,renda_media,faturamento,potencial
160,355030251,A. E. Carvalho,São Paulo,SP,94034,12668,6853,9836,7487,14535,21549,10598,10508,0,253,2197,4368,6681,7011,2247,5670,1501,NaN,NaN


### Aplicar transformações

Avaliar NAs e outros caracteres em X1_prod, para aplicar as mesmas transformações já aplicadas em X1_trainval e X1_test

In [343]:
X1_prod.isna().sum()

codigo             0
bairro             0
cidade             0
estado             0
populacao          0
pop_ate9           0
pop_de10a14        0
pop_de15a19        0
pop_de20a24        0
pop_de25a34        0
pop_de35a49        0
pop_de50a59        0
pop_mais_de60      0
domicilios_a1      0
domicilios_a2      0
domicilios_b1      0
domicilios_b2      0
domicilios_c1      0
domicilios_c2      0
domicilios_d       0
domicilios_e       0
renda_media        0
faturamento      296
potencial        296
dtype: int64

Preencher com zero os 3 registros com '-' em renda_media.

In [341]:
X1_prod.loc[~X1_prod.renda_media.str.isnumeric(), 'renda_media']

232    -
361    -
376    -
Name: renda_media, dtype: object

In [342]:
X1_prod.loc[~X1_prod.renda_media.str.isnumeric(), 'renda_media'] = 0

Aplicar as mesmas transformações nos dados do df_rj para df_sp.

Derivar features.

In [344]:
X1_prod["pop_alvo"] = X1_prod["pop_de25a34"] + X1_prod["pop_de35a49"]
X1_prod["domicilios_alvo"] = X1_prod["domicilios_a1"] + X1_prod["domicilios_a2"] + X1_prod["domicilios_b1"] + X1_prod["domicilios_b2"]

Aplicar a reescala no dataset de produção.

In [348]:
X1_prod[features_scaling] = scaler1.transform(X1_prod[features_scaling])
X1_prod.head(1)

,codigo,bairro,cidade,estado,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,renda_media,faturamento,potencial,pop_alvo,domicilios_alvo
160,355030251,A. E. Carvalho,São Paulo,SP,0.1406,0.1031,0.1223,0.1575,0.1234,0.1685,0.2043,0.1271,0.1148,0.0000,0.0217,0.1135,0.2057,0.2017,0.2719,0.4534,0.3822,0.0134,NaN,NaN,0.1882,0.1436


In [350]:
X1_test.head(1)

,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c2,domicilios_d,domicilios_e,renda_media
110,0.0121,0.0122,0.0148,0.0146,0.0000,0.0080,0.0171,0.0266,0.0406,0.0373,0.0577,0.0164


Manter mesmas features de X1_trainval

In [352]:
X1_trainval.columns

Index(['pop_ate9', 'pop_de10a14', 'pop_de15a19', 'pop_de20a24',
       'domicilios_a1', 'domicilios_a2', 'domicilios_b1', 'domicilios_b2',
       'domicilios_c2', 'domicilios_d', 'domicilios_e', 'renda_media'],
      dtype='object')

In [354]:
X1_prod = X1_prod[['pop_ate9', 'pop_de10a14', 'pop_de15a19', 'pop_de20a24',
       'domicilios_a1', 'domicilios_a2', 'domicilios_b1', 'domicilios_b2',
       'domicilios_c2', 'domicilios_d', 'domicilios_e', 'renda_media']]
X1_prod.head(1)

,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c2,domicilios_d,domicilios_e,renda_media
160,0.1031,0.1223,0.1575,0.1234,0.0000,0.0217,0.1135,0.2057,0.2719,0.4534,0.3822,0.0134


### Prever dados de SP

Utilizar o modelo final tunado já utilizado em X1_test, para prever dados de produção.

In [368]:
knn_yhat_final = knn_reg.predict(X1_prod)

Unificar a previsão com o dataset SP original. 

In [371]:
df_sp_raw['faturamento'] = knn_yhat_final

O dataset SP com as com as 296 previsões de faturamento por bairro será exportado, para ser complementado com as análises 2 e 3.

In [374]:
print(df_sp_raw.shape)
df_sp_raw.head(3)

(296, 24)


,codigo,bairro,cidade,estado,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,renda_media,faturamento,potencial
160,355030251,A. E. Carvalho,São Paulo,SP,94034,12668,6853,9836,7487,14535,21549,10598,10508,0,253,2197,4368,6681,7011,2247,5670,1501,239223.5000,NaN
161,35503020,Aclimação,São Paulo,SP,32791,2297,1017,2096,2197,5341,7281,4917,7645,1413,1734,3704,2351,1946,827,291,1617,5920,1416872.5000,NaN
162,355030285,Adventista,São Paulo,SP,104193,15070,7343,10631,8657,17749,23364,11567,9812,0,0,1423,4875,8595,10082,3111,5776,1284,239223.5000,NaN


In [375]:
df_sp_raw.to_csv('../data/interim/df_sp_an1_v2_done.csv')